In [1]:
from pathlib import Path
import pandas as pd
from dateutil.relativedelta import relativedelta
import plotly.express as px

ruta = Path(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\data\data.xlsx")
df = pd.read_excel(
    ruta, sheet_name="horizonte",
    parse_dates=["Inicio", "Fin"],
    engine="openpyxl"
).rename(columns=str.strip)

fecha_corte_visual = pd.to_datetime("2027-12-31")

df["Fin_recortado"] = df.apply(
    lambda r: fecha_corte_visual if (r["Fin"] > fecha_corte_visual and (r["Fin"] - r["Inicio"]).days > 730)
    else r["Fin"], axis=1
)

df.columns

Index(['Macroetapa', 'Etapa', 'Inicio', 'Fin', 'dias', 'Fin_recortado'], dtype='object')

In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dateutil.relativedelta import relativedelta
from plotly.colors import qualitative

# ---------- 1. PREPARACIÓN ----------
df["Inicio"] = pd.to_datetime(df["Inicio"])
df["Fin"]    = pd.to_datetime(df["Fin"])

df["Duracion_str"] = df.apply(
    lambda r: f"{relativedelta(r['Fin'], r['Inicio']).years} a "
              f"{relativedelta(r['Fin'], r['Inicio']).months} m "
              f"{relativedelta(r['Fin'], r['Inicio']).days} d", axis=1)

inicio_texto = df["Inicio"].min().strftime("%d %b %Y")
fin_texto    = df["Fin"].max().strftime("%d %b %Y")
x_max = df["Fin"].max()  # Solo lo usas si quieres una referencia temporal



# Columna que une Macro + Etapa (respeta tu orden original)
df["Jerarquia"] = df["Macroetapa"] + " – " + df["Etapa"]
y_order = df["Jerarquia"].tolist()                    # orden exacto

# Paleta: colores distintos para cada ETAPA
unique_etapas = df["Etapa"].unique()
paleta = qualitative.Alphabet                      # 26 tonos bien diferenciados
color_map = {e: paleta[i % len(paleta)] for i, e in enumerate(unique_etapas)}

# ---------- 2. GANTT CON PLOTLY EXPRESS ----------
fig = px.timeline(
    df,
    x_start="Inicio",
    x_end="Fin",
    y="Etapa",
    color="Etapa",
    color_discrete_map=color_map,
    category_orders={"Jerarquia": y_order},
    hover_data={
        "Macroetapa": True,
        "Etapa": True,
        "Inicio": "|%d %b %Y",
        "Fin": "|%d %b %Y",
        "Duracion_str": True
    }
)

fig.update_yaxes(
    autorange=True,
    showticklabels=False  # ← esto elimina los nombres del eje Y
)

fig.update_layout(
    height=None,
    autosize=True,
    title="Horiznte de Evaluación",
    title_x=0.5,
    yaxis_visible=False,
    font=dict(color="#d0d0d0"),
    margin=dict(t=60, r=20, l=40, b=40),    
    plot_bgcolor="#0d1117",
    paper_bgcolor="#0d1117",
)



# ---------- 3. (OPCIONAL) SOMBREADO POR MACRO-ETAPA ----------
# Agrega una banda de color suave detrás de cada grupo
y_positions = {y: i for i, y in enumerate(y_order)}          # índice vertical
for macro in df["Macroetapa"].unique():
    subset = df[df["Macroetapa"] == macro]
    y_vals = subset["Jerarquia"].tolist()
    top    = y_positions[y_vals[0]] + 0.5
    bottom = y_positions[y_vals[-1]] - 0.5
    fig.add_shape(
        type="rect",
        x0=df["Inicio"].min(), x1=df["Fin"].max(),
        y0=bottom, y1=top,
        fillcolor="rgba(255,255,255,0.04)", line_width=0,
        layer="below"
    )

# ---------- 4. EXPORTAR ----------
fig.write_html(
    r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\docs\plots\horizonte_evaluacion.html",
    full_html=True,
    include_plotlyjs="cdn",
    config={"responsive": True}
)


fig.show()


In [6]:
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import qualitative, sequential
from pathlib import Path

# ─── PALETAS FIJAS PARA TODA LA TESIS ───────────────────────
PALETA_CATEGORICA = qualitative.Safe      # para categorías (etapas, niveles, regiones)
PALETA_NUMERICA   = sequential.Viridis    # para mapas, proporciones, ratios

# ─── 1. CARGA ────────────────────────────────────────────────
df = pd.read_excel(
    Path(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\data\data.xlsx"),
    sheet_name="demandacp"
)

# ─── 2. PIVOTEO Y ORDEN DE GRADOS ────────────────────────────
df["grado"] = df["grado"].astype(str) + "° grado"
orden = sorted(df["grado"].unique())
df_pivot = df.pivot(index="año", columns="grado", values="total").fillna(0)
df_pivot = df_pivot[orden]

# ─── 3. CENTRADO (CAMPANA) ───────────────────────────────────
totales = df_pivot.sum(axis=1)
bottoms = -totales / 2

# ─── 4. PALETA DE COLORES COHERENTE ──────────────────────────
color_map = {g: PALETA_CATEGORICA[i % len(PALETA_CATEGORICA)] for i, g in enumerate(orden)}

# ─── 5. CREACIÓN DEL GRÁFICO ─────────────────────────────────
fig = go.Figure()

for grado in orden:
    valores = df_pivot[grado]
    fig.add_bar(
        x=df_pivot.index,
        y=valores,
        base=bottoms,
        name=str(grado),
        marker_color=color_map[grado],
        text=valores,
        textposition="inside",
        textfont=dict(color="white", size=14)
    )
    bottoms += valores

# ─── 6. LÍNEAS VERTICALES POR AÑO ────────────────────────────
for año in df_pivot.index:
    fig.add_shape(
        type="line",
        x0=año, x1=año,
        y0=-totales.max() / 2,
        y1=totales.max() / 2,
        line=dict(color="rgba(255,255,255,0.1)", width=10),
        layer="below"
    )

# ─── 7. ESTILO GENERAL ───────────────────────────────────────
fig.update_layout(
    title="Demanda efectiva con proyecto por grado académico",
    title_x=0.5,
    barmode="stack",
    xaxis=dict(tickmode='linear', dtick=1),
    yaxis_visible=False,
    xaxis_visible=True,
    yaxis_showticklabels=False,
    plot_bgcolor="#0d1117",
    paper_bgcolor="#0d1117",
    font=dict(color="#c9d1d9"),
    autosize=True,
    height=None,  # ← evita height fijo
    margin=dict(t=60, r=20, l=40, b=40),
    legend_title="Grado"
)

# ─── 8. EXPORTACIÓN A HTML ───────────────────────────────────
fig.write_html(r"Z:\02_Maestría UNI\I_TALLER DE PROYECTOS II\EXPOSICION\expo2_akira\docs\plots\demanda_efectiva_cp.html", full_html=True,
               include_plotlyjs="cdn",
               config={"responsive": True})

fig.show()
